In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when, dayofweek, year, month, quarter
from pyspark.sql.types import IntegerType, StringType, StructType, StructField
from datetime import datetime, timedelta
import os

# Iniciando uma SparkSession com Delta Lake
spark = SparkSession.builder \
    .appName("BronzeLayer") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .getOrCreate()

# Adicionando configurações S3 diretamente na sessão Spark
spark._jsc.hadoopConfiguration().set("fs.s3a.access.key", os.getenv("AWS_ACCESS_KEY"))
spark._jsc.hadoopConfiguration().set("fs.s3a.secret.key", os.getenv("AWS_SECRET_KEY"))
spark._jsc.hadoopConfiguration().set("fs.s3a.endpoint", "s3.amazonaws.com")
spark._jsc.hadoopConfiguration().set("fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")

# Configurações do bucket S3
bronze_bucket = "s3a://engenharia-dados-satc-bronze-bucket"
silver_bucket = "s3a://engenharia-dados-satc-silver-bucket"

# Carregar arquivos Delta da camada bronze
localizacoes = spark.read.format("delta").load(f'{bronze_bucket}/localizacoes')
produtos = spark.read.format("delta").load(f'{bronze_bucket}/dadosprodutos')
pedidos = spark.read.format("delta").load(f'{bronze_bucket}/pedidos')
clientes = spark.read.format("delta").load(f'{bronze_bucket}/clientes')
vendedores = spark.read.format("delta").load(f'{bronze_bucket}/vendedores')
pagamentos = spark.read.format("delta").load(f'{bronze_bucket}/pagamentos')
dadospedidos = spark.read.format("delta").load(f'{bronze_bucket}/dadospedidos')

# Renomear colunas
clientes = clientes.withColumnRenamed('idgeralcliente', 'idcliente')
produtos = produtos.withColumnRenamed('id_produto', 'idproduto')

### CRIA DIM_DATA ###

# Gerar uma lista de datas de 2015 até hoje + 1 dia
end_date = datetime.now() + timedelta(days=1)
dates = [end_date - timedelta(days=x) for x in range((end_date - datetime(2015, 1, 1)).days + 1)]

# Criar um DataFrame com as datas
schema = StructType([
    StructField('dt_data', StringType(), True)
])
df = spark.createDataFrame([(d.strftime('%Y-%m-%d'),) for d in dates], schema=schema)

df = df.withColumn('dt_diasemana', dayofweek(col('dt_data')))
df = df.withColumn('dt_diasemana_ord', dayofweek(col('dt_data')))
df = df.withColumn('dt_mesano', col('dt_data').substr(0, 7))
df = df.withColumn('dt_mesano_ord', (year(col('dt_data')) - 2015) * 12 + month(col('dt_data')))
df = df.withColumn('dt_trimestre', quarter(col('dt_data')))
df = df.withColumn('dt_trimestre_ord', (year(col('dt_data')) - 2015) * 4 + quarter(col('dt_data')))
df = df.withColumn('dt_ano', year(col('dt_data')))
df = df.withColumn('sk_data', year(col('dt_data')))

# Reorganizar as colunas conforme o formato especificado
datas = df.select('sk_data', 'dt_data', 'dt_diasemana', 'dt_diasemana_ord', 'dt_mesano', 'dt_mesano_ord', 'dt_trimestre', 'dt_trimestre_ord', 'dt_ano')

### CRIA PEDIDOS ###

temp = pedidos.join(dadospedidos, on='idpedido', how='left') \
    .join(pagamentos, on='idpedido', how='left') \
    .join(vendedores, on='idvendedor', how='left') \
    .join(clientes, on='idcliente', how='left') \
    .join(produtos, on='idproduto', how='left')

temp.show()
temp.filter(temp['cepcliente'].isNotNull()).show()

### SALVAR TABELAS NA CAMADA SILVER ###

temp.write.format("delta").mode('overwrite').save(f'{silver_bucket}/pedidos')
localizacoes.write.format("delta").mode('overwrite').save(f'{silver_bucket}/localizacoes')
datas.write.format("delta").mode('overwrite').save(f'{silver_bucket}/dim_data')


+--------------------+--------------------+--------------------+--------------------+------+-------------------+------+-----+------------+-------------------+-------------------+-------------------+-------------------+--------------------+-----------+-----+--------+------+-----------+--------------+----------+--------------------+------------+-----------------+----------------+------+--------------+---------+----------+
|           idproduto|           idcliente|          idvendedor|            idpedido|iditem|              prazo| preco|frete|statuspedido|         datacompra|     data_aprovacao| datatransportadora|        datacliente|dataestimada_entrega|  pagamento|tipos|parcelas| valor|cepvendedor|idunicocliente|cepcliente|           categoria|tamanho_nome|tamanho_descricao|quantidade_fotos|peso_g|comprimento_cm|altura_cm|largura_cm|
+--------------------+--------------------+--------------------+--------------------+------+-------------------+------+-----+------------+--------------